This notebook demonstrates how to search for conserved domains and replicate the searching results from the [NCBI/CDD](https://www.ncbi.nlm.nih.gov/cdd/) website.


In [1]:
import io
import os
import pandas as pd

from Bio.Blast import NCBIXML
from Bio.Blast.Record import Alignment, HSP
from Bio.Blast.Record import Blast as BlastResult
from Bio.Blast.Applications import NcbirpstblastnCommandline

# print usage and version info about rpsblast (for nucleotide)
version_info = NcbirpstblastnCommandline(version=True)()[0]
print(version_info)


rpstblastn: 2.10.1+
 Package: blast 2.10.1, build May 12 2020 13:06:02



In [2]:
# path to CDD (conserved domain database) and related info
CDD_DIR_PATH = os.path.abspath('../data/interim/CDD/Cdd')
CD_TRACK_PATH = '../data/interim/CDD/cd_track.txt'
CD_TRACK_HEADER = [
    'accession',
    'name',
    'id',
    'parent',
    'accession_root',
    'version',
    'live',
    'release',
    'redundant',
    'date',
    'time',
]

# default parameters for rpsblast (replicate the web search results)
RPSBLAST_KWARGS = {
    'db': CDD_DIR_PATH,
    'seg': 'no',
    'outfmt': 5,
    'evalue': 0.01,
    'comp_based_stats': '1',
}

# use Myo7b gene as an example for rpsblast
# sequence source: http://www.informatics.jax.org/sequence/marker/MGI:107709
NUCLEOTIDE_SEQ_PATH = './examples/conserved_domain_search/AXOC01000064.fna'

# paths for conserved domain results
CD_PATH = f'./examples/conserved_domain_search/conserved_domains'
CD_XML_PATH, CD_CSV_PATH = f'{CD_PATH}.xml', f'{CD_PATH}.csv'


In [3]:
# same rpsblast config as the online conserved domain search
# based from the README file in CDD FTP server
# https://ftp.ncbi.nih.gov/pub/mmdb/cdd/README

rpsblast_cmd = NcbirpstblastnCommandline(
    query=NUCLEOTIDE_SEQ_PATH,
    **RPSBLAST_KWARGS,
)
cd_xml, rpsblast_cmd_error_msg = rpsblast_cmd()
with open(CD_XML_PATH, 'w+') as f:
    f.write(cd_xml)


In [4]:
rpsblast_result: BlastResult = NCBIXML.read(io.StringIO(cd_xml))
print(f'There are {len(rpsblast_result.alignments)} rpsblast/rpstblastn alignments ...')

alignment_header = (
    # 'title'
    'id',
    'name',
    'accession',
    'score',
    'e-value',
    'start',
    'end',
    'length',
    'frame',
    'description',
    # 'gaps',
)
alignment_list = []
_alignment: Alignment
for _alignment in rpsblast_result.alignments:

    # _title = (_alignment.title[:32] + '...') \
    #     if len(_alignment.title) > 32 else _alignment.title
    _hit_id = _alignment.hit_id
    _hit_def = (_alignment.hit_def[:32] + '...') \
        if len(_alignment.hit_def) > 32 else _alignment.hit_def

    # check out the HSP documents on the returned fields
    # https://biopython.org/DIST/docs/api/Bio.Blast.Record.HSP-class.html
    _hsp: HSP
    for _hsp in _alignment.hsps:
        _accession, _name, _description = \
            tuple(str(_alignment.hit_def).split(',', maxsplit=2))
        _name =(_name[:8] + '...') if len(_name) > 10 else _name
        _description =(_description[:48] + '...') \
            if len(_description) > 50 else _description
        alignment_list.append([
            # _title,
            int(_hit_id.replace('gnl|CDD|', '')),
            _name,
            _accession,
            _hsp.score,
            _hsp.expect,
            _hsp.query_start,
            _hsp.query_end,
            _hsp.align_length,
            _hsp.frame[0],  # reading frame of the query
            _description,
            # _hsp.gaps,
        ])

alignment_df = pd.DataFrame(alignment_list, columns=alignment_header)
alignment_df.to_csv(CD_CSV_PATH)
print(alignment_df.head(32).drop(columns=['score', 'frame', 'length'])
      .to_markdown(index=False) + '\n...')


There are 218 rpsblast/rpstblastn alignments ...
|     id | name       | accession   |      e-value |   start |   end | description                                        |
|-------:|:-----------|:------------|-------------:|--------:|------:|:---------------------------------------------------|
| 181942 | nhaB       | PRK09547    | 0            |   18715 | 20244 | sodium/proton antiporter NhaB.                     |
| 179607 | umuC       | PRK03609    | 0            |   16714 | 17979 | translesion error-prone DNA polymerase V subuni... |
| 225609 | NhaB       | COG3067     | 0            |   18706 | 20244 | Na+/H+ antiporter [Inorganic ion transport and ... |
| 129856 | Na+/H+_... | TIGR00774   | 0            |   18706 | 20244 | Na+/H+ antiporter NhaB.  These proteins are mem... |
| 283984 | NhaB       | pfam06450   | 0            |   18709 | 20244 | Bacterial Na+/H+ antiporter B (NhaB).  This fam... |
| 182756 | PRK10818   | PRK10818    | 7.85428e-178 |   10761 | 11570 | septum site-

In [5]:
'''
Column descriptions (for CD track info):

Acc = conserved domain model accession number (e.g., pfam09006)

ShortName = first 10 characters of domain model's short name,
        in this case, Surfac_D-t, for Surfac_D-trimer.

PSSMID = unique identifier for the position specific scoring matrix
        (e.g., as the pfam09006 domain model has evolved, it has had
        three PSSMs, with IDs 72424, 87766, and 90442, respectively).

        If there are any changes in the protein sequence alignment
        of a domain model (for example, the addition/deletion of
        member protein sequences or changes in the span of aligned residues),
        or if there are changes in the interpretation of the alignment,
        a new PSSM will be calculated. In that case, it will receive
        a new PSSM ID, although the accession number of the conserved
        domain model will remain the same.

        If only the domain model description or other annotations have
        changed, but the PSSM did not change, the version of the model
        will be incremented but the the PSSM ID will remain the same,
        as it did for version 1 and 2 of pfam09006, both of which had
        the PSSM ID 72424.

Root =  if the domain model is NCBI-curated, the "Root" column will
        show the accession number of the parent node of the curated
        domain hierarchy.  If the domain hierarchy contains only a
        single node, the value in the "Root" column will be the same
        as that in the "Acc" column.  The values will also be the same
        if the accession listed in the first column is the parent node
        of a multi-level hierarchy.

Version = version number of that particular domain model

Lv =         indicates the current live version of the record:
        1 = live status;
        0 = dead, earlier version.

Rl =         indicates whether the domain model version has been
             released into the public database. This is a flag
             NCBI uses for internal data tracking.
             For most domain models, the value will be
             1= released, which means at some point the model was
             live in the database. Occasionally a value of "0" might
             appear, primarily for ncbi-curated models.  This indicates
             a newer version of a model is in preparation at NCBI and
             will be released in the future.

ER =         Expendable or redundant models; value in this column can be:
             0 = non-expendable or not redundant
             1 = expendable or redundant; indicates a model that has been
             removed from the default "cdd" search set because the
             information in it is represented in another domain model.

Time =         date and time on which the model was last updated in the
        internal conserved domain tracking database.
'''

cd_track_df = pd.read_csv(
    CD_TRACK_PATH,
    sep='\s+',
    header=None,
    index_col=None,
    skiprows=4,
    names=CD_TRACK_HEADER,
)

_cd_track_cd_len = len(cd_track_df)

# delete the "dead" rows and keep the useful columns only
cd_track_df = cd_track_df.loc[cd_track_df['live'] == 1]
print(f'There are {len(cd_track_df)}/{_cd_track_cd_len} (live/total) PSSMs in CDD ...')
print(cd_track_df.head(32)[['accession', 'name', 'id', 'parent', 'accession_root']].
      to_markdown())

There are 16181/16181 (live/total) PSSMs in CDD ...
|    | accession   | name           |     id | parent   | accession_root   |
|---:|:------------|:---------------|-------:|:---------|:-----------------|
|  0 | cd00001     | PTS_IIB_man    | 237975 | nan      | cd00001          |
|  1 | cd00002     | YbaK_deacylase | 237976 | cd04332  | cd04332          |
|  2 | cd00003     | PNPsynthase    | 237977 | nan      | cd00003          |
|  3 | cd00004     | Sortase        | 320674 | nan      | cd00004          |
|  4 | cd00005     | CBM9_like_1    | 187674 | cd00241  | cd00241          |
|  5 | cd00006     | PTS_IIA_man    | 237978 | nan      | cd00006          |
|  6 | cd00008     | PIN_53EXO-like | 350199 | cd09853  | cd09852          |
|  7 | cd00009     | AAA            |  99707 | nan      | cd00009          |
|  8 | cd00010     | AAI_LTSS       | 237980 | nan      | cd00010          |
|  9 | cd00011     | BAR_Arfapti... | 153270 | cd07307  | cd07307          |
| 10 | cd00012     | NBD

In [6]:
_unique_cd_root = cd_track_df['accession_root'].unique()
print(f'There are {len(_unique_cd_root)} unique CD roots in CDD ...')

# merge the CD track info into (rpsblast) alignment dataframe
alignment_cd_track_df = alignment_df.merge(
    right=cd_track_df[['id', 'accession_root']],
    how='left',
    on='id',
)

# fill in the short names and accession
# _def = alignment_cd_track_df['def'].str.split(',', n=2 ,expand=True)
# _accessions, _short_names = _def[0], _def[1]
# alignment_cd_track_df['accession'] = _accessions
# alignment_cd_track_df['short_name'] = _short_names

# fill in the root if it does not exist
# for non-NCBI-curated conserved domains, there is no root in CD track
# alignment_cd_track_df['root'].fillna(
#     alignment_cd_track_df['accession'],
#     inplace=True,
# )

print(alignment_cd_track_df.head(32)[
          ['id', 'name', 'accession', 'accession_root', 'e-value', 'start', 'end', 'description']].to_markdown())


There are 1270 unique CD roots in CDD ...
|    |     id | name       | accession   | accession_root   |      e-value |   start |   end | description                                        |
|---:|-------:|:-----------|:------------|:-----------------|-------------:|--------:|------:|:---------------------------------------------------|
|  0 | 181942 | nhaB       | PRK09547    | nan              | 0            |   18715 | 20244 | sodium/proton antiporter NhaB.                     |
|  1 | 179607 | umuC       | PRK03609    | nan              | 0            |   16714 | 17979 | translesion error-prone DNA polymerase V subuni... |
|  2 | 225609 | NhaB       | COG3067     | nan              | 0            |   18706 | 20244 | Na+/H+ antiporter [Inorganic ion transport and ... |
|  3 | 129856 | Na+/H+_... | TIGR00774   | nan              | 0            |   18706 | 20244 | Na+/H+ antiporter NhaB.  These proteins are mem... |
|  4 | 283984 | NhaB       | pfam06450   | nan              | 0       

In [ ]:
# TODO: The binding sites and other annotated information is in cddannot.dat from the CDD FTP server
# Note that the info is only available for NCBI-curated domains
